In [2]:
import numpy as np
import pandas as pd

# FUNZIONI DEL MODELLO

def qA_eps(qA0, gamma, eps):
    return qA0 + gamma * eps

def cA_eps(cA0, beta_c, eps):
    return cA0 - beta_c * eps

def equilibrium_prices(qA0, qB, gamma, eps, cA0, beta_c, cB, t, theta_bar):
    qA = qA_eps(qA0, gamma, eps)
    cA = cA_eps(cA0, beta_c, eps)

    pA = (2*cA + cB)/3 + t + theta_bar*(qA - qB)/3
    pB = (cA + 2*cB)/3 + t - theta_bar*(qA - qB)/3
    return pA, pB, qA, cA

def equilibrium_quantities(pA, pB, qA, qB, cA, cB, t, theta_bar):
    qA_dem = (cB - cA + 3*t + theta_bar*(qA - qB)) / (6*t)
    qB_dem = 1 - qA_dem
    return qA_dem, qB_dem

def profits(pA, pB, qA_dem, qB_dem, cA, cB, eps, delta):
    piA = (pA - cA) * qA_dem - (delta*eps/2)**2
    piB = (pB - cB) * qB_dem
    return piA, piB

def optimal_eps(qA0, qB, cA0, beta_c, gamma, t, theta_bar, delta, cB):
   

    A = cB - cA0 + 3*t + theta_bar*(qA0 - qB)
    theta = beta_c + theta_bar*gamma

    denom = (theta**2)/(9*t) - (delta**2)/2
    # caso degenerato: niente soluzione interna → ε* = 0
    if np.isclose(denom, 0.0):
        return 0.0

    eps_star = - (theta * A / (9*t)) / denom
    # vincolo ε ≥ 0
    return max(eps_star, 0.0)


# simulazioniene di uno scenario con ε* calcolato
def simulate_scenario_with_eps_star(name, params):
    

    eps_star = optimal_eps(
        qA0   = params["qA0"],
        qB    = params["qB"],
        cA0   = params["cA"],
        beta_c= params["beta_c"],
        gamma = params["gamma"],
        t     = params["t"],
        theta_bar = params["theta"],
        delta = params["delta"],
        cB    = params["cB"]
    )

    eps_values = [0.0, eps_star]

    results = []
    for eps in eps_values:
        pA, pB, qA, cA = equilibrium_prices(
            params["qA0"], params["qB"], params["gamma"], eps,
            params["cA"], params["beta_c"], params["cB"],
            params["t"], params["theta"]
        )

        qA_d, qB_d = equilibrium_quantities(
            pA, pB, qA, params["qB"], cA, params["cB"],
            params["t"], params["theta"]
        )

        piA, piB = profits(pA, pB, qA_d, qB_d, cA, params["cB"], eps, params["delta"])

        results.append([eps, pA, pB, qA_d, qB_d, piA, piB])

    df = pd.DataFrame(results, columns=["eps","pA","pB","qA","qB","piA","piB"])
    df.name = name + f" (eps_star = {eps_star:.4f})"
    return df

# PARAMETRI BASE

base_params = {
    "qA0": 1.0,
    "qB": 1.0,
    "gamma": 0.0,
    "beta_c": 0.0,
    "cA": 1.0,
    "cB": 1.0,
    "t": 0.8,
    "theta": 0.9,
    "delta": 1.2
}

# DEFINIZIONE DEI 5 SCENARI

# S1 – Nessun investimento ESG (parametri ESG nulli)
params_S1 = base_params.copy()
params_S1["gamma"] = 0.0
params_S1["beta_c"] = 0.0

# S2 – ESG riduce i costi (beta_c > 0, gamma = 0)
params_S2 = base_params.copy()
params_S2["beta_c"] = 0.7
params_S2["gamma"] = 0.0

# S3 – ESG aumenta il valore del prodotto (gamma > 0, beta_c = 0)
params_S3 = base_params.copy()
params_S3["gamma"] = 0.7
params_S3["beta_c"] = 0.0

# S4 – Combinazione positiva: qualità ↑, costi ↓
params_S4 = base_params.copy()
params_S4["gamma"] = 0.6
params_S4["beta_c"] = 0.6

# S5 – Effetti contrastanti: qualità ↑, ma costi ↑ (beta_c < 0)
params_S5 = base_params.copy()
params_S5["gamma"] = 0.6
params_S5["beta_c"] = -0.6



# COSTRUZIONE DELLE 5 TABELLE (CON ε*)

df_S1 = simulate_scenario_with_eps_star("S1 – No ESG", params_S1)
df_S2 = simulate_scenario_with_eps_star("S2 – ESG reduces cost", params_S2)
df_S3 = simulate_scenario_with_eps_star("S3 – ESG increases value", params_S3)
df_S4 = simulate_scenario_with_eps_star("S4 – Mixed positive effects", params_S4)
df_S5 = simulate_scenario_with_eps_star("S5 – Conflicting effects", params_S5)

dfs = [df_S1, df_S2, df_S3, df_S4, df_S5]

# FUNZIONE PER IL CONSUMER WELFARE

def consumer_welfare(theta_bar, qA, qB, pA, pB, t, x_star):
    
    termA = (theta_bar*qA - pA) * x_star - (t/2)*(x_star**2)
    termB = (theta_bar*qB - pB) * (1 - x_star) - (t/2)*((1 - x_star)**2)
    return termA + termB

# AGGIORNAMENTO DELLA FUNZIONE CHE ESTRAE LA RIGA DI EQUILIBRIO

def extract_equilibrium_row(df, params):
    row = df.iloc[1]  # riga con eps*
    eps = row["eps"]

    # variabili di equilibrio
    pA, pB = row["pA"], row["pB"]
    qA, qB = row["qA"], row["qB"]
    x_star = qA  # identico al consumatore indifferente

    # Consumer welfare
    CW = consumer_welfare(
        theta_bar = params["theta"],
        qA = qA,
        qB = params["qB"],
        pA = pA,
        pB = pB,
        t = params["t"],
        x_star = x_star
    )

    return {
        "eps_star": eps,
        "pA": pA,
        "pB": pB,
        "qA": qA,
        "qB": qB,
        "piA": row["piA"],
        "piB": row["piB"],
        "CW": CW
    }


# COSTRUZIONE DELLA TABELLA RIASSUNTIVA (CON CW)

summary = pd.DataFrame([
    {"Scenario": "S1 – No ESG", **extract_equilibrium_row(df_S1, params_S1)},
    {"Scenario": "S2 – ESG reduces cost", **extract_equilibrium_row(df_S2, params_S2)},
    {"Scenario": "S3 – ESG increases value", **extract_equilibrium_row(df_S3, params_S3)},
    {"Scenario": "S4 – Mixed positive effects", **extract_equilibrium_row(df_S4, params_S4)},
    {"Scenario": "S5 – Conflicting effects", **extract_equilibrium_row(df_S5, params_S5)},
])

summary = summary[
    ["Scenario", "eps_star", "pA", "pB", "qA", "qB", "piA", "piB", "CW"]
]
CW_baseline = summary.loc[summary["Scenario"]=="S1 – No ESG", "CW"].values[0]
summary["CW_norm"] = summary["CW"] - CW_baseline
summary_2 = summary[
    ["Scenario", "eps_star", "pA", "pB", "qA", "qB", "piA", "piB", "CW_norm"]
]
print(summary_2.round(4))

                      Scenario  eps_star      pA      pB      qA      qB  \
0                  S1 – No ESG    0.0000  1.8000  1.8000  0.5000  0.5000   
1        S2 – ESG reduces cost    0.3579  1.6330  1.7165  0.5522  0.4478   
2     S3 – ESG increases value    0.3158  1.8663  1.7337  0.5415  0.4585   
3  S4 – Mixed positive effects    0.7044  1.6450  1.5323  0.6673  0.3327   
4     S5 – Conflicting effects    0.0000  1.8000  1.8000  0.5000  0.5000   

      piA     piB  CW_norm  
0  0.4000  0.4000   0.0000  
1  0.4418  0.3208   0.1299  
2  0.4332  0.3364  -0.0053  
3  0.5338  0.1771   0.1953  
4  0.4000  0.4000   0.0000  


In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize_scalar

class DixitStiglitzESG:
    def __init__(self,
                 sigma=4.2,
                 Y=1,
                 N=10,
                 cA=1.0,
                 cB=1.0,
                 qA0=1.0,
                 qB=1.0,
                 gamma=0.7,
                 betac=0.7,
                 delta=1.0):

        self.sigma = sigma
        self.Y = Y
        self.N = N

        self.cA_baseline = cA
        self.cB = cB

        self.qA0 = qA0
        self.qB = qB

        self.gamma = gamma
        self.betac = betac
        self.delta = delta
 
    #   QUALITÀ
    
    def quality_A(self, eps, scenario):
        if scenario in [1, 2]:
            return self.qA0
        else:
            return self.qA0 * (1 + self.gamma * eps)

    #   COSTI
    
    def cost_A(self, eps, scenario):
        if scenario == 1 or scenario == 3:
            return self.cA_baseline
        elif scenario == 2 or scenario == 4:
            return self.cA_baseline * (1 - self.betac * eps)
        elif scenario == 5:
            return self.cA_baseline * (1 + self.betac * eps)

    #   PREZZI
   
    def price(self, c):
        return (self.sigma / (self.sigma - 1)) * c

    #   PRICE INDEX
    
    def price_index(self, pA, qA):
        pB = self.price(self.cB)
        termA = qA**(self.sigma - 1) * pA**(1 - self.sigma)
        termB = self.qB**(self.sigma - 1) * pB**(1 - self.sigma)
        return (termA + (self.N - 1) * termB)**(1 / (1 - self.sigma))
   
    #   DOMANDA
  
    def demand_A(self, pA, qA, Pq):
        return self.Y * (pA / Pq)**(-self.sigma) * (qA**(self.sigma - 1))

    def demand_B(self):
        
        pB = self.price(self.cB)
        PB = (self.N * (self.qB / pB)**self.sigma)**(1/self.sigma)
        return (self.qB/pB)**self.sigma * self.Y / PB**(self.sigma - 1)

    #   PROFITTO NETTO
    
    def profit_A(self, eps, scenario):
        qA = self.quality_A(eps, scenario)
        cA = self.cost_A(eps, scenario)
        pA = self.price(cA)
        Pq = self.price_index(pA, qA)
        qA_d = self.demand_A(pA, qA, Pq)

        operating_profit = (pA - cA) * qA_d
        investment_cost = (self.delta**2 / 4) * eps**2
        return operating_profit - investment_cost


    #   EPS ENDOGENO

    def optimal_epsilon(self, scenario, bounds=(0, 2)):
        objective = lambda eps: -self.profit_A(eps, scenario)
        solution = minimize_scalar(objective, bounds=bounds, method="bounded")
        eps_star = solution.x
        return eps_star, self.profit_A(eps_star, scenario)

    #   SIMULAZIONE SCENARIO
   
    def simulate_scenario(self, scenario):
        eps_star, pi_star = self.optimal_epsilon(scenario)

        qA = self.quality_A(eps_star, scenario)
        cA = self.cost_A(eps_star, scenario)
        pA = self.price(cA)

        Pq = self.price_index(pA, qA)
        qA_d = self.demand_A(pA, qA, Pq)

        # Brown equilibrium
        xB = self.demand_B()
        pB = self.price(self.cB)
        piB = (pB - self.cB) * xB

        # Market share
        share_A = qA_d / (qA_d + (self.N - 1) * xB)

        # Better welfare
        welfare = qA * qA_d + (self.N - 1) * self.qB * xB

        return {
            "scenario": scenario,
            "epsilon_star": eps_star,
            "profit_A": pi_star,
            "profit_normalized": pi_star / piB,
            "profit_as_share_of_income": pi_star / self.Y,
            "quality_A": qA,
            "cost_A": cA,
            "price_A": pA,
            "quantity_A": qA_d,
            "market_share_A": share_A,
            "price_index": Pq,
            "welfare_total": welfare
        }
    
    #   TUTTI GLI SCENARI



    def simulate_all_scenarios(self):
        return pd.DataFrame([self.simulate_scenario(s) for s in range(1,6)])
    

#   RUN del codice

model = DixitStiglitzESG()
df = model.simulate_all_scenarios()
print(df.round(4))


   scenario  epsilon_star  profit_A  profit_normalized  \
0         1        0.0000    0.0152             0.3694   
1         2        0.0698    0.0162             0.3944   
2         3        0.0626    0.0161             0.3919   
3         4        0.2097    0.0214             0.5185   
4         5        0.0000    0.0152             0.3694   

   profit_as_share_of_income  quality_A  cost_A  price_A  quantity_A  \
0                     0.0152     1.0000  1.0000   1.3125      0.0487   
1                     0.0162     1.0000  0.9511   1.2484      0.0588   
2                     0.0161     1.0438  1.0000   1.3125      0.0548   
3                     0.0214     1.1468  0.8532   1.1198      0.1214   
4                     0.0152     1.0000  1.0000   1.3125      0.0487   

   market_share_A  price_index  welfare_total  
0          0.0394       0.6391         1.2351  
1          0.0472       0.6357         1.2452  
2          0.0441       0.6362         1.2436  
3          0.0928       0.

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize_scalar

class DixitStiglitzESG:
    def __init__(self,
                 sigma=4.2,
                 Y=1,
                 N=10,
                 cA=1.0,
                 cB=1.0,
                 qA0=1.0,
                 qB=1.0,
                 gamma=0.7,
                 betac=0.7,
                 delta=1.0):

        self.sigma = sigma
        self.Y = Y
        self.N = N

        self.cA_baseline = cA
        self.cB = cB

        self.qA0 = qA0
        self.qB = qB

        self.gamma = gamma
        self.betac = betac
        self.delta = delta
 
    #   QUALITÀ
    
    def quality_A(self, eps, scenario):
        if scenario in [1, 2]:
            return self.qA0
        else:
            return self.qA0 * (1 + self.gamma * eps)

    #   COSTI
    
    def cost_A(self, eps, scenario):
        if scenario == 1 or scenario == 3:
            return self.cA_baseline
        elif scenario == 2 or scenario == 4:
            return self.cA_baseline * (1 - self.betac * eps)
        elif scenario == 5:
            return self.cA_baseline * (1 + self.betac * eps)

    #   PREZZI
   
    def price(self, c):
        return (self.sigma / (self.sigma - 1)) * c

    #   PRICE INDEX
    
    def price_index(self, pA, qA):
        pB = self.price(self.cB)
        termA = qA**(self.sigma - 1) * pA**(1 - self.sigma)
        termB = self.qB**(self.sigma - 1) * pB**(1 - self.sigma)
        return (termA + (self.N - 1) * termB)**(1 / (1 - self.sigma))
   
    #   DOMANDA
  
    def demand_A(self, pA, qA, Pq):
        return self.Y * (pA / Pq)**(-self.sigma) * (qA**(self.sigma - 1))

    def demand_B(self, Pq):
       pB = self.price(self.cB)
       return self.Y * (pB / Pq)**(-self.sigma) * (self.qB**(self.sigma - 1))

    #   PROFITTO NETTO
    
    def profit_A(self, eps, scenario):
        qA = self.quality_A(eps, scenario)
        cA = self.cost_A(eps, scenario)
        pA = self.price(cA)
        Pq = self.price_index(pA, qA)
        qA_d = self.demand_A(pA, qA, Pq)

        operating_profit = (pA - cA) * qA_d
        investment_cost = (self.delta**2 / 4) * eps**2
        return operating_profit - investment_cost


    #   EPS ENDOGENO

    def optimal_epsilon(self, scenario, bounds=(0, 2)):
        objective = lambda eps: -self.profit_A(eps, scenario)
        solution = minimize_scalar(objective, bounds=bounds, method="bounded")
        eps_star = solution.x
        return eps_star, self.profit_A(eps_star, scenario)

    def welfare(self, qA, qB, price_A, price_B, Pq):
    # Aggiungi la relazione tra domanda e varietà, considerando N
        welfare_A = qA * (price_A / Pq)  # Welfare dato dalla qualità e quantità di A
        welfare_B = qB * (price_B / Pq)  # Welfare dato dalla qualità e quantità di B
        welfare_total = welfare_A + (self.N - 1) * welfare_B  # concorrenza
        return welfare_total
    #   SIMULAZIONE SCENARIO
   
    def simulate_scenario(self, scenario):
        eps_star, pi_star = self.optimal_epsilon(scenario)

        qA = self.quality_A(eps_star, scenario)
        cA = self.cost_A(eps_star, scenario)
        pA = self.price(cA)

        Pq = self.price_index(pA, qA)
        qA_d = self.demand_A(pA, qA, Pq)

        # Brown equilibrium
        xB = self.demand_B(Pq)
        pB = self.price(self.cB)
        piB = (pB - self.cB) * xB

        # Market share
        share_A = qA_d / (qA_d + (self.N - 1) * xB)

        # Better welfare
        welfare = self.welfare(qA, self.qB, pA, pB, Pq)
    

        return {
            "scenario": scenario,
            "epsilon_star": eps_star,
            "profit_A": pi_star,
            "quality_A": qA,
            "price_A": pA,
            "quantity_A": qA_d,
            "market_share_A": share_A,
            "price_index": Pq,
            "welfare_total": welfare
        }
    
    #   TUTTI GLI SCENARI



    def simulate_all_scenarios(self):
        return pd.DataFrame([self.simulate_scenario(s) for s in range(1,6)])
    

#   RUN del codice

model = DixitStiglitzESG()
df = model.simulate_all_scenarios()
print(df.round(4))


   scenario  epsilon_star  profit_A  quality_A  price_A  quantity_A  \
0         1        0.0000    0.0152     1.0000   1.3125      0.0487   
1         2        0.0698    0.0162     1.0000   1.2484      0.0588   
2         3        0.0626    0.0161     1.0438   1.3125      0.0548   
3         4        0.2097    0.0214     1.1468   1.1198      0.1214   
4         5        0.0000    0.0152     1.0000   1.3125      0.0487   

   market_share_A  price_index  welfare_total  
0          0.1000       0.6391        20.5353  
1          0.1206       0.6357        20.5453  
2          0.1130       0.6362        20.7196  
3          0.2512       0.6106        21.4502  
4          0.1000       0.6391        20.5353  
